In [1]:
import torch
import torch_geometric

import identify_x86_graph

In [2]:
print(torch.__version__)

1.13.0


In [7]:
model = torch.jit.load('model_jit.pt', map_location='cpu')

print(model)

RecursiveScriptModule(
  original_name=IdentifyModel
  (model): RecursiveScriptModule(
    original_name=Sequential_d2a63c
    (module_0): RecursiveScriptModule(original_name=Embedding)
    (module_1): RecursiveScriptModule(original_name=Embedding)
    (module_3): RecursiveScriptModule(
      original_name=RGCNConvJittable_d28bee
      (aggr_module): RecursiveScriptModule(original_name=MeanAggregation)
    )
    (module_4): RecursiveScriptModule(original_name=ReLU)
    (module_5): RecursiveScriptModule(
      original_name=RGCNConvJittable_d2941e
      (aggr_module): RecursiveScriptModule(original_name=MeanAggregation)
    )
    (module_6): RecursiveScriptModule(original_name=ReLU)
    (module_7): RecursiveScriptModule(
      original_name=RGCNConvJittable_d29c7f
      (aggr_module): RecursiveScriptModule(original_name=MeanAggregation)
    )
    (module_8): RecursiveScriptModule(original_name=ReLU)
    (module_9): RecursiveScriptModule(
      original_name=RGCNConvJittable_d2a4f1
     

In [8]:
G = identify_x86_graph.load_graph('data/raw/debian/buster/binutils-i686-linux-gnu/usr_bin_i686-linux-gnu-addr2line.graph')

G

Data(num_nodes=25188, x_code=[25188], x_size=[25188], y=[25188], num_edges=278940, edge_index=[2, 278940], edge_type=[278940])

In [9]:
example_input = (G.x_code, G.x_size, G.edge_index, G.edge_type)

with torch.no_grad():
    example_output = model(*example_input)

print(example_output)

tensor([[ 0.8675, -0.8305],
        [ 0.8675, -0.8305],
        [ 0.8675, -0.8305],
        ...,
        [ 0.8675, -0.8305],
        [ 0.8675, -0.8305],
        [ 0.8675, -0.8305]])


In [10]:
torch.onnx.export(
    model = model,
    args = (G.x_code, G.x_size, G.edge_index, G.edge_type),
    f = 'model.onnx',
    input_names = ['x_code', 'x_size', 'edge_index', 'edge_type'],
    operator_export_type=torch.onnx.OperatorExportTypes.ONNX,
    opset_version = 18,
    verbose = True,
    dynamic_axes = None,
    export_modules_as_functions = False,
)

/home/dcnick3/.conda/envs/ml/lib/python3.10/site-packages/torch/onnx/utils.py:823: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f7c02552ca0>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")
